In [5]:
import pandas as pd
import numpy as np

In [79]:

#read from flat excel waste per category percentage
df1_waste_cate_per = pd.read_excel('../source/waste_cate_per.xlsx', 'Sheet 1') 
df2_waste_cate_per = pd.read_excel('../source/waste_cate_per.xlsx', 'Sheet 2')

#read from flat excel waste per category capita
df1_waste_cate_cap = pd.read_excel('../source/waste_cate_capita.xlsx', 'Sheet 1') 
df2_waste_cate_cap = pd.read_excel('../source/waste_cate_capita.xlsx', 'Sheet 2')
#read from flat excel food waste 
df1_food_waste = pd.read_excel('../source/food_waste.xlsx','Sheet 1')
df2_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 2')
df3_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 3')
df4_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 4')


#read from bigquery csv
df_bq_keywords = pd.read_csv('../source/bq_keywords_food_waste.csv')


#read from API csv
df_api_12 = pd.read_csv('../source/api_data12.csv')
df_api_12_3 = pd.read_csv('../source/api_data12_3.csv')

df_top_fruit_veg_wasted = pd.read_csv('../source/top_fruit_veg_wasted.csv')

df_national_fruit = pd.read_csv('../source/national_fruit.csv')
df_country_region = pd.read_csv('../source/country_region.csv')




C:\Users\krant\AppData\Local\Temp\ipykernel_8908\3324755166.py:16: DtypeWarning: Columns (9,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bq_keywords = pd.read_csv('../source/bq_keywords_food_waste.csv')


In [194]:

eu_country = ['Albania',
'Austria',
'Belgium',
'Bosnia and Herzegovina',
'Bulgaria',
'Croatia',
'Cyprus',
'Czechia',
'Denmark',
'Estonia',
'Finland',
'France',
'Germany',
'Greece',
'Hungary',
'Iceland',
'Ireland',
'Italy',
'Latvia',
'Lithuania',
'Luxembourg',
'Malta',
'Montenegro',
'Netherlands (Kingdom of the)',
'North Macedonia',
'Norway',
'Poland',
'Portugal',
'Romania',
'Serbia',
'Slovakia',
'Slovenia',
'Spain',
'Sweden',
'Switzerland',
'TÃ¼rkiye',
'United Kingdom of Great Britain and Northern Ireland','Netherlands Antilles [former]','Kosovo',
'Belgium',
'Bulgaria',
'Czechia',
'Denmark',
'Germany',
'Estonia',
'Ireland',
'Greece',
'Spain',
'France',
'Croatia',
'Italy',
'Cyprus',
'Latvia',
'Lithuania',
'Luxembourg',
'Hungary',
'Malta',
'Netherlands',
'Austria',
'Poland',
'Portugal',
'Romania',
'Slovenia',
'Slovakia',
'Finland',
'Sweden',
'Iceland',
'Norway',
'Switzerland',
'United Kingdom',
'Bosnia and Herzegovina',
'Montenegro',
'North Macedonia',
'Albania',
'Serbia',
'TÃ¼rkiye',
'Kosovo*','Türkiye']




In [147]:

#read from flat excel waste per category
 

# Slice rows from index 8 to 48 of df1_waste_cate_per and reset the index, dropping the old index
df_waste_per = df1_waste_cate_per[11:49].reset_index(drop=True)

# Rename the columns of df_waste_per using the values from the 9th row of df1_waste_cate_per
df_waste_per = df_waste_per.rename(columns=df1_waste_cate_per.iloc[7])

# Add a new column named 'year' to df_waste_per and set all values in that column to 2020
df_waste_per['year'] = 2020

# Slice rows from index 8 to 48 of df2_waste_cate_per and reset the index, dropping the old index
df_waste2 = df2_waste_cate_per[11:49].reset_index(drop=True)

# Rename the columns of df_waste2 using the values from the 9th row of df2_waste_cate_per
df_waste2 = df_waste2.rename(columns=df2_waste_cate_per.iloc[7])

# Add a new column named 'year' to df_waste2 and set all values in that column to 2021
df_waste2['year'] = 2021

# Concatenate df_waste_per and df_waste2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_waste_per = pd.concat([df_waste_per, df_waste2]).reset_index(drop=True)

df_waste_per

# First, find the column names that are NaN
nan_columns = df_waste_per.columns[df_waste_per.columns.isna()]

# Then drop these columns
df_waste_per_c = df_waste_per.drop(columns=nan_columns)

df_waste_per_c = df_waste_per_c.rename(columns={'COICOP (Labels)':'country'})
df_waste_per_c.columns = [col.lower().replace(' ', '_') for col in df_waste_per_c.columns]


df_waste_per_c['id'] = df_waste_per_c.apply(lambda row: f'{row["country"].lower()}{row["year"]}', axis=1)
# Assuming 'df_waste_per_c' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'  # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_waste_per_c[column_to_move]

# Drop the column from its original position
df_waste_per_c = df_waste_per_c.drop(columns=[column_to_move])

# Insert the column at the first position
df_waste_per_c.insert(0, column_to_move, moved_column)

df_waste_per_c.columns
# df_waste_cap_c.to_csv('../source/check.csv',index=False)

# Mapping original column names to shortened names
column_mapping = {
    'alcoholic_beverages,_tobacco_and_narcotics': 'substances',
    'clothing_and_footwear': 'clothing',
    'housing,_water,_electricity,_gas_and_other_fuels': 'house_utilities',
    'furnishings,_household_equipment_and_routine_household_maintenance': 'house_maintenance',
    'recreation_and_culture': 'recreation_culture',
    'restaurants_and_hotels': 'restaurants_hotels',
    'non-alcoholic_beverages': 'beverages',
    'miscellaneous_goods_and_services': 'miscellaneous'
}

# Rename columns using the mapping
df_waste_per_c = df_waste_per_c.rename(columns=column_mapping)


In [145]:
#read from flat excel waste per category
 

# Slice rows from index 8 to 48 of df1_waste_cate_cap and reset the index, dropping the old index
df_waste_cap = df1_waste_cate_cap[11:49].reset_index(drop=True)

# Rename the columns of df_waste_cap using the values from the 9th row of df1_waste_cate_cap
df_waste_cap = df_waste_cap.rename(columns=df1_waste_cate_cap.iloc[7])

# Add a new column named 'year' to df_waste_cap and set all values in that column to 2020
df_waste_cap['year'] = 2020

# Slice rows from index 8 to 48 of df2_waste_cate_cap and reset the index, dropping the old index
df_waste2_cap = df2_waste_cate_cap[11:49].reset_index(drop=True)

# Rename the columns of df_waste2_cap using the values from the 9th row of df2_waste_cate_cap
df_waste2_cap = df_waste2_cap.rename(columns=df2_waste_cate_cap.iloc[7])

# Add a new column named 'year' to df_waste2_cap and set all values in that column to 2021
df_waste2_cap['year'] = 2021

# Concatenate df_waste_cap and df_waste2_cap vertically, stack them on top of each other, and reset the index, dropping the old index
df_waste_cap = pd.concat([df_waste_cap, df_waste2_cap]).reset_index(drop=True)

df_waste_cap

# First, find the column names that are NaN
nan_columns = df_waste_cap.columns[df_waste_cap.columns.isna()]

# Then drop these columns
df_waste_cap_c = df_waste_cap.drop(columns=nan_columns)

df_waste_cap_c = df_waste_cap_c.rename(columns={'COICOP (Labels)':'country'})
df_waste_cap_c.columns = [col.lower().replace(' ', '_') for col in df_waste_cap_c.columns]


df_waste_cap_c['id'] = df_waste_cap_c.apply(lambda row: f'{row["country"].lower()}{row["year"]}', axis=1)
# Assuming 'df_waste_cap_c' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'  # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_waste_cap_c[column_to_move]

# Drop the column from its original position
df_waste_cap_c = df_waste_cap_c.drop(columns=[column_to_move])

# Insert the column at the first position
df_waste_cap_c.insert(0, column_to_move, moved_column)



# Mapping original column names to shortened names
column_mapping = {
    'alcoholic_beverages,_tobacco_and_narcotics': 'substances',
    'clothing_and_footwear': 'clothing',
    'housing,_water,_electricity,_gas_and_other_fuels': 'house_utilities',
    'furnishings,_household_equipment_and_routine_household_maintenance': 'house_maintenance',
    'recreation_and_culture': 'recreation_culture',
    'restaurants_and_hotels': 'restaurants_hotels',
    'non-alcoholic_beverages': 'beverages',
    'miscellaneous_goods_and_services': 'miscellaneous'
}

# Rename columns using the mapping
df_waste_cap_c = df_waste_per_c.rename(columns=column_mapping)
# df_waste_cap_c.to_csv('../source/check.csv',index=False)



In [144]:
# Slice rows from index 12 to 38 of df1_food_waste and reset the index, dropping the old index
df_kilo_capita = df1_food_waste[12:39].reset_index(drop=True)

# Rename the columns of df_kilo_capita using the values from the 9th row of df1_food_waste
df_kilo_capita = df_kilo_capita.rename(columns=df1_food_waste.iloc[9])

# Add a new column named 'year' to df_kilo_capita and set all values in that column to 2020
df_kilo_capita['year'] = 2020

# Slice rows from index 12 to 38 of df2_food_waste and reset the index, dropping the old index
df_kilo_capita2 = df2_food_waste[12:39].reset_index(drop=True)

# Rename the columns of df_kilo_capita2 using the values from the 9th row of df2_food_waste
df_kilo_capita2 = df_kilo_capita2.rename(columns=df2_food_waste.iloc[9])

# Add a new column named 'year' to df_kilo_capita2 and set all values in that column to 2021
df_kilo_capita2['year'] = 2021

# Concatenate df_kilo_capita and df_kilo_capita2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_kilo_capita = pd.concat([df_kilo_capita, df_kilo_capita2]).reset_index(drop=True)

# First, find the column names that are NaN
nan_columns = df_kilo_capita.columns[df_kilo_capita.columns.isna()]

# Then drop these columns
df_kilo_capita_c = df_kilo_capita.drop(columns=nan_columns)
df_kilo_capita_c


df_kilo_capita_c = df_kilo_capita_c.rename(columns={'NACE_R2 (Labels)':'country'})
# Convert column names to lowercase, replace spaces with underscores, and replace parentheses with underscores
df_kilo_capita_c.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_kilo_capita_c.columns]


df_kilo_capita_c['id'] = df_kilo_capita_c.apply(lambda row: f'{row["country"].lower()}{row["year"]}', axis=1)
# Assuming 'df_kilo_capita_c' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'   # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_kilo_capita_c[column_to_move]

# Drop the column from its original position
df_kilo_capita_c = df_kilo_capita_c.drop(columns=[column_to_move])

# Insert the column at the first position
df_kilo_capita_c.insert(0, column_to_move, moved_column)
# Insert the column at the first position


# Define the mapping of old names to new names
column_mapping = {
    'primary_production_of_food_-_agriculture,_fishing_and_aquaculture': 'food_production',
    'manufacture_of_food_products_and_beverages': 'food_manufacture',
    'retail_and_other_distribution_of_food': 'food_distribution',
    'restaurants_and_food_services': 'food_services',
    'total_activities_by_households': 'households'
}

# Use the replace function to rename columns
df_kilo_capita_c.rename(columns=column_mapping, inplace=True)
#cleaned



In [148]:
# Slice rows from index 10 to 38 of df3_food_waste and reset the index, dropping the old index
df_tonne = df3_food_waste[12:39].reset_index(drop=True)

# Rename the columns of df_tonne using the values from the 9th row of df3_food_waste
df_tonne = df_tonne.rename(columns=df3_food_waste.iloc[9])

# Add a new column named 'year' to df_tonne and set all values in that column to 2020
df_tonne['year'] = 2020

# Slice rows from index 10 to 38 of df4_food_waste and reset the index, dropping the old index
df_tonne2 = df4_food_waste[12:39].reset_index(drop=True)

# Rename the columns of df_tonne2 using the values from the 9th row of df4_food_waste
df_tonne2 = df_tonne2.rename(columns=df4_food_waste.iloc[9])

# Add a new column named 'year' to df_tonne2 and set all values in that column to 2021
df_tonne2['year'] = 2021

# Concatenate df_tonne and df_tonne2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_tonne = pd.concat([df_tonne, df_tonne2]).reset_index(drop=True)


# First, find the column names that are NaN
nan_columns = df_tonne.columns[df_tonne.columns.isna()]

# Then drop these columns
df_tonne_c = df_tonne.drop(columns=nan_columns)






df_tonne_c = df_tonne_c.rename(columns={'NACE_R2 (Labels)':'country'})
# Convert column names to lowercase, replace spaces with underscores, and replace parentheses with underscores
df_tonne_c.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_tonne_c.columns]


df_tonne_c['id'] = df_tonne_c.apply(lambda row: f'{row["country"].lower()}{row["year"]}', axis=1)
# Assuming 'df_tonne_c' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'   # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_tonne_c[column_to_move]

# Drop the column from its original position
df_tonne_c = df_tonne_c.drop(columns=[column_to_move])

# Insert the column at the first position
df_tonne_c.insert(0, column_to_move, moved_column)

#claned
df_tonne_c.columns.value_counts()

# Define the mapping of old names to new names
column_mapping = {
    'primary_production_of_food_-_agriculture,_fishing_and_aquaculture': 'food_production',
    'manufacture_of_food_products_and_beverages': 'food_manufacture',
    'retail_and_other_distribution_of_food': 'food_distribution',
    'restaurants_and_food_services': 'food_services',
    'total_activities_by_households': 'total_household_activities'
}

# Use the replace function to rename columns
df_tonne_c.rename(columns=column_mapping, inplace=True)



In [150]:

#read from text file 
df_composition = pd.read_csv('../source/food_composition.csv')

df_composition
# Convert column names to lowercase, replace spaces with underscores, and replace parentheses with underscores
df_composition.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_composition.columns]


#claned ??
# df_composition.to_csv('../source/check.csv',index=False)


In [152]:

df_bq_keywords_eu = df_bq_keywords[df_bq_keywords['geoareaname'].isin(eu_country)].reset_index(drop=True)

#DROP EMPTY COLUMN
df_bq_keywords_eu = df_bq_keywords_eu.drop(columns=['cities','education_level','freq','hazard_type','ihr_capacity','level_status','location','migratory_status','mode_of_transportation',
'name_of_international_institution','name_of_non_communicable_disease','sex','tariff_regime_status','type_of_mobile_technology','type_of_occupation','type_of_skill','type_of_speed'])

df_bq_keywords_eu['geoareaname'] = df_bq_keywords_eu['geoareaname'].replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom")


df_bq_keywords_eu['id'] = df_bq_keywords_eu.apply(lambda row: f'{row["geoareaname"].lower()}{row["timeperiod"]}', axis=1)
# Assuming 'df_bq_keywords_eu' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'   # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_bq_keywords_eu[column_to_move]

# Drop the column from its original position
df_bq_keywords_eu = df_bq_keywords_eu.drop(columns=[column_to_move])

# Insert the column at the first position
df_bq_keywords_eu.insert(0, column_to_move, moved_column)
df_bq_keywords_eu = df_bq_keywords_eu.rename(columns={'geoareaname':'country'})
df_bq_keywords_eu = df_bq_keywords_eu.rename(columns={'timeperiod':'year'})

#claned
df_bq_keywords_eu


,id,goal,target,indicator,seriescode,seriesdescription,geoareacode,country,year,value,time_detail,source,footnote,nature,age,bounds,type_of_product,units
0,bulgaria2015,3,3.5,3.5.2,SH_ALC_CONSPT,Alcohol consumption per capita (aged 15 years ...,100,Bulgaria,2015,12.38205,NaN,WHO Global Information System on Alcohol and H...,NaN,G,15+,NaN,NaN,LTPUREAL
1,france2000,3,3.5,3.5.2,SH_ALC_CONSPT,Alcohol consumption per capita (aged 15 years ...,250,France,2000,13.82911,NaN,WHO Global Information System on Alcohol and H...,NaN,G,15+,NaN,NaN,LTPUREAL
2,lithuania2015,3,3.5,3.5.2,SH_ALC_CONSPT,Alcohol consumption per capita (aged 15 years ...,440,Lithuania,2015,15.47114,NaN,WHO Global Information System on Alcohol and H...,NaN,G,15+,NaN,NaN,LTPUREAL
3,hungary2005,3,3.5,3.5.2,SH_ALC_CONSPT,Alcohol consumption per capita (aged 15 years ...,348,Hungary,2005,14.54439,NaN,WHO Global Information System on Alcohol and H...,NaN,G,15+,NaN,NaN,LTPUREAL
4,austria2005,3,3.5,3.5.2,SH_ALC_CONSPT,Alcohol consumption per capita (aged 15 years ...,40,Austria,2005,12.81204,NaN,WHO Global Information System on Alcohol and H...,NaN,G,15+,NaN,NaN,LTPUREAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59060,germany2005,2,2.2,2.2.2,SH_STA_WASTE,Proportion of children moderately or severely ...,276,Germany,2005,1.00000,2003-06 (2005),Die Verbreitung von Übergewicht und Adipositas...,NaN,CA,<5Y,MP,NaN,PER_POP_U5
59061,serbia2005,2,2.2,2.2.2,SH_STA_WASTE,Proportion of children moderately or severely ...,688,Serbia,2005,4.50000,2005-06 (2005),Republic of Serbia multiple indicator cluster ...,NaN,CA,<5Y,MP,NaN,PER_POP_U5
59062,montenegro2005,2,2.2,2.2.2,SH_STA_WASTE,Proportion of children moderately or severely ...,499,Montenegro,2005,4.20000,2005-06 (2005),Montenegro multiple indicator cluster survey 2...,NaN,CA,<5Y,MP,NaN,PER_POP_U5
59063,albania2009,2,2.2,2.2.2,SH_STA_WASTE,Proportion of children moderately or severely ...,8,Albania,2009,9.40000,2008-09 (2009),Albania demographic and health survey 2008-09....,NaN,CA,<5Y,MP,NaN,PER_POP_U5


In [154]:


df_api_12['goal'] = df_api_12['goal'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12['target'] = df_api_12['target'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12['indicator'] = df_api_12['indicator'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12['attributes'] = df_api_12['attributes'].apply(lambda x: x.replace('{','').replace('}','').replace("'", ''))
df_api_12['dimensions'] = df_api_12['dimensions'].apply(lambda x: x.replace('{','').replace('}','').replace("'", ''))
df_api_12['footnotes'] = df_api_12['footnotes'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))

# df_api_12['value'] = df_api_12['value'].apply(lambda x: float(x))
df_api_12['value'] = df_api_12['value'].apply(lambda x:int(x))

df_12_eu = df_api_12[df_api_12['geoAreaName'].isin(eu_country)].reset_index(drop=True)

df_12_eu.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_12_eu.columns]

#DROP EMPTY COLUMN
df_12_eu = df_12_eu.drop(columns=['time_detail','timecoverage','upperbound','lowerbound','geoinfourl'])

df_12_eu['geoareaname'] = df_12_eu['geoareaname'].replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom")

df_12_eu['geoareaname'] = df_12_eu['geoareaname'].replace("Netherlands (Kingdom of the)", "Netherlands")

df_12_eu['timeperiodstart'] = df_12_eu['timeperiodstart'].astype(int)


df_12_eu['id'] = df_12_eu.apply(lambda row: f'{row["geoareaname"].lower()}{row["timeperiodstart"]}', axis=1)
# Assuming 'df_12_eu' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'   # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_12_eu[column_to_move]

# Drop the column from its original position
df_12_eu = df_12_eu.drop(columns=[column_to_move])

# Insert the column at the first position
df_12_eu.insert(0, column_to_move, moved_column)

df_12_eu = df_12_eu.rename(columns={'geoareaname':'country'})
df_12_eu = df_12_eu.rename(columns={'timeperiodstart':'year'})

#claned
df_12_eu





,id,goal,target,indicator,series,seriesdescription,seriescount,geoareacode,country,year,value,valuetype,baseperiod,source,footnotes,attributes,dimensions
0,albania2019,12,12.1,12.1.1,SG_SCP_CNTRY,Countries with sustainable consumption and pro...,158,8,Albania,2019,1,Float,NaN,Ministry of Tourism and Environment,,"Nature: C, Units: NUMBER, Observation Status: A",Reporting Type: G
1,belgium2021,12,12.1,12.1.1,SG_SCP_CNTRY,Countries with sustainable consumption and pro...,158,56,Belgium,2021,1,Float,NaN,"Federal Planning Bureau (FPB), Brussels Enviro...",,"Nature: C, Units: NUMBER, Observation Status: A",Reporting Type: G
2,bulgaria2021,12,12.1,12.1.1,SG_SCP_CNTRY,Countries with sustainable consumption and pro...,158,100,Bulgaria,2021,1,Float,NaN,Ministry of Environment and Water,,"Nature: C, Units: NUMBER, Observation Status: A",Reporting Type: G
3,croatia2021,12,12.1,12.1.1,SG_SCP_CNTRY,Countries with sustainable consumption and pro...,158,191,Croatia,2021,1,Float,NaN,Ministry for Economy and Sustainable Development,,"Nature: C, Units: NUMBER, Observation Status: A",Reporting Type: G
4,czechia2019,12,12.1,12.1.1,SG_SCP_CNTRY,Countries with sustainable consumption and pro...,158,203,Czechia,2019,1,Float,NaN,Ministry of Environment,,"Nature: C, Units: NUMBER, Observation Status: A",Reporting Type: G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,bulgaria2022,"12, 8","12.2, 8.4","12.2.2, 8.4.2",EN_MAT_DOMCMPT,"Domestic material consumption, by type of raw ...",94414,100,Bulgaria,2022,43890522,Float,NaN,Eurostat,Eurostat estimate,"Nature: C, Units: TONNES, Observation Status: E","Type of product: FOF, Reporting Type: G"
1598,bulgaria2022,"12, 8","12.2, 8.4","12.2.2, 8.4.2",EN_MAT_DOMCMPT,"Domestic material consumption, by type of raw ...",94414,100,Bulgaria,2022,33352696,Float,NaN,Eurostat,Eurostat estimate,"Nature: C, Units: TONNES, Observation Status: E","Type of product: MEO, Reporting Type: G"
1599,bulgaria2022,"12, 8","12.2, 8.4","12.2.2, 8.4.2",EN_MAT_DOMCMPT,"Domestic material consumption, by type of raw ...",94414,100,Bulgaria,2022,25523792597,Float,NaN,Eurostat,,"Nature: C, Units: TONNES, Observation Status: A","Type of product: NFO, Reporting Type: G"
1600,bulgaria2022,"12, 8","12.2, 8.4","12.2.2, 8.4.2",EN_MAT_DOMCMPT,"Domestic material consumption, by type of raw ...",94414,100,Bulgaria,2022,22789912,Float,NaN,Eurostat,Eurostat estimate,"Nature: C, Units: TONNES, Observation Status: E","Type of product: BIM, Reporting Type: G"


In [160]:

df_api_12_3['goal'] = df_api_12_3['goal'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12_3['target'] = df_api_12_3['target'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12_3['indicator'] = df_api_12_3['indicator'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12_3['attributes'] = df_api_12_3['attributes'].apply(lambda x: x.replace('{','').replace('}','').replace("'", ''))
df_api_12_3['dimensions'] = df_api_12_3['dimensions'].apply(lambda x: x.replace('{','').replace('}','').replace("'", ''))
df_api_12_3['footnotes'] = df_api_12_3['footnotes'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_12_3_eu = df_api_12_3[df_api_12_3['geoAreaName'].isin(eu_country)].reset_index(drop=True)
df_12_3_eu.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_12_3_eu.columns]

#DROP EMPTY COLUMN
df_12_3_eu = df_12_3_eu.drop(columns=['time_detail','timecoverage','upperbound','lowerbound','baseperiod','geoinfourl'])



df_12_3_eu['timeperiodstart'] = df_12_3_eu['timeperiodstart'].astype(int)

df_12_3_eu['geoareaname'] = df_12_3_eu['geoareaname'].replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom")

df_12_3_eu['geoareaname'] = df_12_3_eu['geoareaname'].replace("Netherlands (Kingdom of the)", "Netherlands")



df_12_3_eu['id'] = df_12_3_eu.apply(lambda row: f'{row["geoareaname"].lower()}{row["timeperiodstart"]}', axis=1)
# Assuming 'df_12_3_eu' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'   # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_12_3_eu[column_to_move]

# Drop the column from its original position
df_12_3_eu = df_12_3_eu.drop(columns=[column_to_move])

# Insert the column at the first position
df_12_3_eu.insert(0, column_to_move, moved_column)

df_12_3_eu = df_12_3_eu.rename(columns={'geoareaname':'country'})
df_12_3_eu = df_12_3_eu.rename(columns={'timeperiodstart':'year'})





In [162]:
#clean

# df_top_fruit_veg_wasted

In [161]:
df_country_region
df_country_region.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_country_region.columns]
df_country_region


df_country_region['country'] = df_country_region['country'].replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom")

# df_country_region['country'] = df_country_region['geoacountryreaname'].replace("Netherlands (Kingdom of the)", "Netherlands")


In [197]:
# 
df_national_fruit = pd.read_csv('../source/national_fruit.csv')
# df_national_fruit.tail(30)


df_national_fruit = df_national_fruit[df_national_fruit['country'].isin(eu_country)].reset_index(drop=True)


df_national_fruit

,country,common_name
0,Albania,Cherry
1,Austria,Apple
2,Belgium,Apple
3,Bulgaria,Apple
4,Cyprus,Banana
5,Denmark,Strawberries
6,France,Pear
7,Germany,Apple
8,Greece,Olive
9,Hungary,Orange


In [199]:
# display(df_waste_per_c,df_waste_cap_c,
# df_kilo_capita_c,
# df_tonne_c,
# df_composition,
# df_bq_keywords_eu,
# df_12_eu,
# df_12_3_eu,
# df_top_fruit_veg_wasted,
# df_national_fruit,
# df_country_region)

df_waste_per_c.to_csv('../source/df_waste_per_c.csv', index=False)
df_waste_cap_c.to_csv('../source/df_waste_cap_c.csv', index=False)
df_kilo_capita_c.to_csv('../source/df_kilo_capita_c.csv', index=False)
df_tonne_c.to_csv('../source/df_tonne_c.csv', index=False)
df_composition.to_csv('../source/df_composition.csv', index=False)
df_bq_keywords_eu.to_csv('../source/df_bq_keywords_eu.csv', index=False)
df_12_eu.to_csv('../source/df_12_eu.csv', index=False)
df_12_3_eu.to_csv('../source/df_12_3_eu.csv', index=False)
df_top_fruit_veg_wasted.to_csv('../source/df_top_fruit_veg_wasted.csv', index=False)
df_national_fruit.to_csv('../source/df_national_fruit.csv', index=False)
df_country_region.to_csv('../source/df_country_region.csv', index=False)